GOLF SHOT SWING RECOGNITION USING OPTICAL FLOW.

Implementing Gunnar Farneback based Optical Flow for trajectory estimation.


Implementing Morphological Filter on image frame contributing trajectory path.

In [146]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
# Path to folder with videos
videos_folder = './dataset'

# Path to folder where optical flow images will be saved
optical_flow_folder_gf = './optical_flow_images_gf'
optical_flow_folder_lk = './optical_flow_images_lk'

# Loop through all videos in the folder
for video_name in os.listdir(videos_folder):
    
    # Get the label of the video from its name (assuming that the label is the first word in the file name)
    label = video_name.split('_')[0]
    bgSubtractor = cv2.createBackgroundSubtractorMOG2()
    # Open the video file
    video_path = os.path.join(videos_folder, video_name)
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, frame1 = cap.read()
    i = 0
    # Loop through the rest of the frames
    while True:
        
        # Read the next frame
        ret, frame2 = cap.read()
        
        # If there are no more frames, break out of the loop
        if not ret:
            break
        # Convert frames to grayscale
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        # Calculate optical flow using Farnback algorithm
        flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Convert optical flow to image format
        hsv = np.zeros((frame1.shape[0], frame1.shape[1], 3), dtype=np.uint8)
        
        hsv[...,1] = 255
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)

        optical_flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Save the optical flow image with the corresponding label in the optical flow folder
        optical_flow_name = f'{label}_{str(i).zfill(4)}.png'
        i=i+1
        optical_flow_path = os.path.join(optical_flow_folder_gf, optical_flow_name)
        cv2.imwrite(optical_flow_path, optical_flow_image)
        
        # Apply background subtraction to frame
        fgMask = bgSubtractor.apply(frame2)

        # Apply morphological operations to reduce noise and fill gaps
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)

        
        # Save the optical flow image with the corresponding label in the optical flow folder
        optical_flow_name = f'{label}_{str(i).zfill(4)}.png'
        i=i+1
        optical_flow_path = os.path.join(optical_flow_folder_lk, optical_flow_name)
        cv2.imwrite(optical_flow_path, fgMask)


        # Update the current frame
        frame1 = frame2
        
    # Release the video capture object
    cap.release()

Define Function for extraction of HOG features from Gunnar Farneback Images.

In [26]:
import cv2
import numpy as np
from skimage.feature import hog


def extract_hog_features(image):
    # define the HOG parameters
    orientations = 9
    pixels_per_cell = (16, 16)
    cells_per_block = (2, 2)
    
    # compute the HOG features for the image
    features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=False, transform_sqrt=True)
    print('\n')
    # return the features as a 1D NumPy array
    return features.ravel()

In [41]:
import csv
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
csv_file_path = './features_back.csv'
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row with feature names
    num_bins = 10
    header = ['Mean', 'Standard Deviation', 'Variance','Label'] + ['Bin {}'.format(i) for i in range(num_bins)] + ['Sum HOG', 'Mean HOG', 'Standard Deviation HOG', 'Variance HOG'] + ['HOG Hist {}'.format(i) for i in range(0,10)]
    writer.writerow(header)

    for image in os.listdir('./optical_flow_images_gf'):
        image_path = os.path.join('./optical_flow_images_gf', image)

        # Load the preprocessed optical flow image
        optical_flow = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Calculate the mean, standard deviation, and variance of the optical flow image
        try:
            mean = np.mean(optical_flow)
            std = np.std(optical_flow)
            var = np.var(optical_flow)
        except:
            continue
        # Calculate the histogram of the optical flow image
        hist, bins = np.histogram(optical_flow, bins=num_bins, range=(-10, 10))
        hist = hist.tolist()
        
        h=np.unique(extract_hog_features(optical_flow))
        sum_hog=np.sum(h)
        mean_hog=np.mean(h)
        std_hog=np.std(h)
        var_hog=np.var(h)
        hog_hist=np.zeros(10)
        for i in h:
            if i>0.0 and i<0.1:
                hog_hist[0]+=1
            elif i>0.1 and i<0.2:
                hog_hist[1]+=1
            elif i>0.2 and i<0.3:
                hog_hist[2]+=1
            elif i>0.3 and i<0.4:
                hog_hist[3]+=1
            elif i>0.4 and i<0.5:
                hog_hist[4]+=1
            elif i>0.5 and i<0.6:
                hog_hist[5]+=1
            elif i>0.6 and i<0.7:
                hog_hist[6]+=1
            elif i>0.7 and i<0.8:
                hog_hist[7]+=1
            elif i>0.8 and i<0.9:
                hog_hist[8]+=1
            else:
                hog_hist[9]+=1
        hog_hist=hog_hist.tolist()        
        
        # Append the features to the CSV file
        writer.writerow([mean, std, var,image.split('_')[0]] + hist + [sum_hog, mean_hog, std_hog, var_hog] + hog_hist )
        # Print the calculated features
        #print('Mean: {:.2f}'.format(mean))
        #print('Standard Deviation: {:.2f}'.format(std))
        #print('Variance: {:.2f}'.format(var))
        #print('Histogram: {}'.format(hist))

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Read CSV file into a Pandas DataFrame
df = pd.read_csv('./features_back.csv')

# Define the columns to be normalized
cols_to_norm = ['Mean', 'Standard Deviation', 'Variance', 'Bin 5', 'Bin 6','Bin 7', 'Bin 8','Bin 9','Sum HOG', 'Mean HOG', 'Standard Deviation HOG', 'HOG Hist 0', 'HOG Hist 1', 'HOG Hist 2', 'HOG Hist 3', 'HOG Hist 4', 'HOG Hist 5','HOG Hist 6','HOG Hist 7','HOG Hist 8','HOG Hist 9']

# Create a scaler object
scaler = MinMaxScaler()

# Normalize the selected columns using the scaler
df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])

df.to_csv('./features_back.csv', index=False)


          Mean  Standard Deviation  Variance   Label  Bin 0  Bin 1  Bin 2  \
0     0.086395            0.296581  0.119031    back      0      0      0   
1     0.050625            0.173118  0.051290    back      0      0      0   
2     0.119815            0.248251  0.089423    back      0      0      0   
3     0.129642            0.341006  0.149754    back      0      0      0   
4     0.060640            0.208397  0.067999    back      0      0      0   
...        ...                 ...       ...     ...    ...    ...    ...   
1053  0.230066            0.320413  0.135095  follow      0      0      0   
1054  0.124572            0.198407  0.063052  follow      0      0      0   
1055  0.160668            0.201009  0.064324  follow      0      0      0   
1056  0.119285            0.169203  0.049566  follow      0      0      0   
1057  0.386801            0.371797  0.173019  follow      0      0      0   

      Bin 3  Bin 4     Bin 5  ...  HOG Hist 0  HOG Hist 1  HOG Hist 2  \
0 

SIFT feature extraction from morphological image.

In [42]:
import csv
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
csv_file_path = './features_back_lk.csv'
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    header = ['SIFT {}'.format(i) for i in range(0,128)]
    writer.writerow(header)
    for image in os.listdir('./optical_flow_images_lk'):
        image_path = os.path.join('./optical_flow_images_lk', image)
        label=image.split('_')[0]
        # Load the preprocessed optical flow image
        optical_flow = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        #initialise sift descriptor
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(optical_flow, None)
        # sift_image = cv2.drawKeypoints(gray, keypoints, img)
        # print(type(descriptors))
        try:
            descriptors=descriptors.T
            # pca = PCA(n_components=0.95)
            # # Fit the transformer to the data and transform the data
            # descriptors = pca.fit_transform(descriptors)
            # # Reshape the descriptors array into a 1D array
            descriptors_1d = descriptors.reshape(-1)
            descriptors_1d = descriptors_1d[:128]
        except:
            descriptors_1d=np.zeros(128)
        # Append the new feature to an existing row
        writer.writerow(descriptors_1d.tolist())
        


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Read CSV file into a Pandas DataFrame
df = pd.read_csv('./features_back_lk.csv')

# Define the columns to be normalized
cols_to_norm = ['SIFT {}'.format(i) for i in range(0,128)]

# Create a scaler object
scaler = MinMaxScaler()

# Normalize the selected columns using the scaler
df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])

df.to_csv('./features_back_lk.csv', index=False)

        SIFT 0    SIFT 1    SIFT 2    SIFT 3    SIFT 4    SIFT 5    SIFT 6  \
0     0.866106  0.632695  0.461859  0.449646  0.549696  0.380122  0.466683   
1     0.610998  0.351353  0.403890  0.703854  0.434921  0.662251  0.280480   
2     0.489854  0.386070  0.580198  0.564362  0.422432  0.535476  0.368421   
3     0.604521  0.361964  0.545194  0.603571  0.408072  0.278031  0.511354   
4     0.601887  0.452448  0.593554  0.625858  0.382814  0.494532  0.723068   
...        ...       ...       ...       ...       ...       ...       ...   
1053  0.579201  0.480422  0.400291  0.560080  0.737868  0.522182  0.526370   
1054  0.545510  0.462191  0.614856  0.390300  0.414835  0.538223  0.435255   
1055  0.554743  0.598361  0.572004  0.601054  0.408230  0.531984  0.379482   
1056  0.541040  0.457218  0.336172  0.560201  0.707961  0.501126  0.371671   
1057  0.459604  0.489164  0.570025  0.486173  0.726231  0.540978  0.285900   

        SIFT 7    SIFT 8    SIFT 9  ...  SIFT 118  SIFT 119  SI

In [8]:
import pandas as pd
import csv

# Define the paths to the CSV files
csv_path_1 = "./features_back.csv"
csv_path_2 = "./features_back_lk.csv"

# Read the CSV files into separate pandas DataFrames
df1 = pd.read_csv(csv_path_1)
df2 = pd.read_csv(csv_path_2)
# print(df1.columns.tolist())
# print(df2.columns.tolist())
# print(df1.loc[0].tolist()+df2.loc[0].tolist())
csv_file_path = './features.csv'
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    header = df1.columns.tolist()+df2.columns.tolist()
    writer.writerow(header)
    row=[]
    for i in range(1058):
        row=df1.loc[i].tolist()+df2.loc[i].tolist()
        writer.writerow(row)

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the CSV file with extracted features
csv_file_path = './features.csv'
df = pd.read_csv(csv_file_path)

# Split the data into training and testing sets
#
df = df.drop('Bin 0',axis=1)
df = df.drop('Bin 1',axis=1)
df = df.drop('Bin 2',axis=1)
df = df.drop('Bin 3',axis=1)
df = df.drop('Bin 4',axis=1)

X = df.drop('Label',axis=1)
y = df['Label']
# print👍

new_col_values = []

# loop through rows of DataFrame
for row in df.iterrows():
    # print(row[1]['Label'])
    if row[1]['Label']=="back":
        new_col_values.append(0)
    elif row[1]['Label']=="down":
        new_col_values.append(1)
    elif row[1]['Label']=="follow":
        new_col_values.append(2)

# add new_col to DataFrame
df['Class'] = new_col_values

df.to_csv(csv_file_path)

# print updated DataFrame
print(df)

          Mean  Standard Deviation   Variance   Label    Bin 5  Bin 6  Bin 7  \
0     0.556674            4.696250  22.054761    back  1094022  31458  15121   
1     0.359380            3.186641  10.154681    back  1114348  23904  10572   
2     0.741009            4.105302  16.853508    back  1072170  38711  16174   
3     0.795212            5.239447  27.451801    back  1078831  33555  14005   
4     0.414621            3.618001  13.089933    back  1111613  22207  11422   
...        ...                 ...        ...     ...      ...    ...    ...   
1053  1.349114            4.987656  24.876710  follow  1002155  53381  30298   
1054  0.767247            3.495845  12.220935  follow  1046373  45812  24532   
1055  0.966338            3.527668  12.444442  follow  1016054  54029  31988   
1056  0.738081            3.138769   9.851868  follow  1043169  51997  26083   
1057  2.213606            5.615943  31.538815  follow   868385  92652  57924   

      Bin 8  Bin 9      Sum HOG  ...   

In [11]:

df = df.drop('Label',axis=1)
X = df.iloc[:, :-1]  # selects all rows (:) and all but the last column (:-1) of the DataFrame df.
Y = df.iloc[:, -1]    #  selects all rows (:) and the last column (-1) of the DataFrame df.

# train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=5)

In [12]:
X

,Mean,Standard Deviation,Variance,Bin 5,Bin 6,Bin 7,Bin 8,Bin 9,Sum HOG,Mean HOG,...,SIFT 118,SIFT 119,SIFT 120,SIFT 121,SIFT 122,SIFT 123,SIFT 124,SIFT 125,SIFT 126,SIFT 127
0,0.556674,4.696250,22.054761,1094022,31458,15121,7711,6220,2814.904485,0.152868,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.359380,3.186641,10.154681,1114348,23904,10572,4861,4056,1880.101450,0.148472,...,-74.772041,53.019558,-91.951920,-35.534462,134.467514,-39.382416,13.225515,5.185749,-25.173859,-2.575677
2,0.741009,4.105302,16.853508,1072170,38711,16174,8883,8404,3193.181075,0.146449,...,119.858444,64.790573,-17.979828,-11.774525,10.488808,39.699814,30.242283,-12.214418,-21.354784,97.238396
3,0.795212,5.239447,27.451801,1078831,33555,14005,9516,9568,2813.931181,0.145536,...,-45.751472,-8.146722,23.674538,16.879667,-29.073320,-32.995296,-8.107094,10.847504,-168.261124,-165.720993
4,0.414621,3.618001,13.089933,1111613,22207,11422,6288,4955,1720.966500,0.143342,...,-28.151375,20.717064,-37.631458,-11.912526,2.186476,-57.171062,12.862970,-19.574232,24.376343,33.111610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1.349114,4.987656,24.876710,1002155,53381,30298,18536,17274,5067.250268,0.138893,...,-5.987559,27.111052,69.208099,-64.069801,0.011178,18.976994,-21.876720,-63.722820,-223.607834,-112.845856
1054,0.767247,3.495845,12.220935,1046373,45812,24532,14780,13416,3089.014868,0.130531,...,8.201757,39.517273,42.473213,-129.972305,40.082336,25.547880,-0.518717,-135.653442,-0.696796,57.745838
1055,0.966338,3.527668,12.444442,1016054,54029,31988,19087,17610,4046.270988,0.134979,...,-127.677567,22.768131,-12.361745,-34.419037,-54.391846,54.337955,-134.791870,68.316101,-37.639439,-42.464138
1056,0.738081,3.138769,9.851868,1043169,51997,26083,13887,12107,3269.506477,0.129949,...,67.561134,24.446341,-24.754156,44.638222,7.681120,13.457706,66.544075,-33.429150,-5.936067,-14.595716


In [13]:
Y

0       0
1       0
2       0
3       0
4       0
       ..
1053    2
1054    2
1055    2
1056    2
1057    2
Name: Class, Length: 1058, dtype: int64

In [14]:
x_train

,Mean,Standard Deviation,Variance,Bin 5,Bin 6,Bin 7,Bin 8,Bin 9,Sum HOG,Mean HOG,...,SIFT 118,SIFT 119,SIFT 120,SIFT 121,SIFT 122,SIFT 123,SIFT 124,SIFT 125,SIFT 126,SIFT 127
281,0.395749,2.993559,8.961397,692638,4804,3773,3559,3741,718.828264,0.142033,...,-14.364077,17.185566,4.719944,-20.514751,-6.963824,27.971216,-21.066498,21.718739,1.722710,-0.640827
830,0.565631,6.114240,37.383935,697335,4999,2594,1542,1686,579.192828,0.143471,...,-23.141712,35.656250,-25.227655,-6.477972,8.991025,-188.709259,-4.774416,79.631378,38.242477,21.802788
683,1.031885,6.530226,42.643851,677264,7990,5239,3734,3830,1232.412288,0.140510,...,41.050419,-0.057453,50.055229,-63.717213,-14.580037,87.122253,-4.349473,-3.669290,-42.261986,10.519012
871,0.645607,6.193820,38.363402,696932,5960,2094,1191,2059,496.504969,0.138264,...,-2.855215,36.042580,34.075413,-18.208763,-24.695837,-13.331373,60.803856,-43.332256,30.979496,-24.738859
595,0.149964,1.770950,3.136264,706812,2856,2157,1090,834,440.807715,0.168633,...,-5.605232,-20.277100,-15.782651,13.718862,-11.134361,-0.968225,-13.858082,-135.757111,-42.307270,9.925239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,2.362716,6.395596,40.903648,832399,128542,67458,38195,33420,8949.271374,0.126329,...,83.593719,-68.629028,-52.677101,-28.887522,25.744452,15.880880,-2.844508,-392.439697,-108.001328,-45.398540
73,0.979023,5.981757,35.781414,1086071,29942,11212,5695,5855,2800.138127,0.150618,...,153.133026,45.206825,79.063889,-89.260727,6.035251,-36.186443,-164.663254,-163.252563,53.850796,-97.443893
998,0.666427,3.606549,13.007192,1066480,43049,19533,10962,9229,3251.677323,0.141790,...,-43.079197,-108.933044,-36.067825,-56.453537,1.692349,-110.791862,36.061924,-60.707550,127.437180,242.351257
206,0.382268,4.382471,19.206053,697982,5193,2961,1828,2474,587.808965,0.144923,...,28.080463,-20.961449,-39.358562,51.183037,-43.236012,78.299339,-35.319405,-26.463886,-39.023876,-40.996368


In [15]:
y_train


281     0
830     1
683     1
871     1
595     0
       ..
1032    2
73      0
998     2
206     0
867     1
Name: Class, Length: 846, dtype: int64

In [16]:
x_test

,Mean,Standard Deviation,Variance,Bin 5,Bin 6,Bin 7,Bin 8,Bin 9,Sum HOG,Mean HOG,...,SIFT 118,SIFT 119,SIFT 120,SIFT 121,SIFT 122,SIFT 123,SIFT 124,SIFT 125,SIFT 126,SIFT 127
763,1.051461,8.376339,70.163054,684665,5443,3950,4236,3962,764.386099,0.128167,...,-36.567074,-14.384775,-11.734731,37.541889,13.553376,-0.292490,-14.866241,-24.228504,14.047499,-1.397409
299,0.388549,2.913033,8.485760,693077,4127,3494,3348,4036,622.158726,0.139435,...,6.850658,9.915338,-9.978570,-20.811613,5.730747,-7.889675,-14.599580,-18.032177,16.570675,4.651615
897,1.000225,6.148837,37.808200,1097417,15494,7525,5695,7542,1818.978238,0.140625,...,-191.719376,-260.093048,42.887104,-141.701385,-129.961090,-232.116409,232.042786,87.147858,241.042679,176.877167
341,0.280698,2.752521,7.576374,697887,5483,3004,2249,3177,598.177976,0.146720,...,2.614320,-40.131428,-15.332307,0.684482,4.942315,-18.190737,-36.530193,-15.180065,24.401747,4.227867
40,0.522675,3.299020,10.883532,1111035,17915,10108,5928,4734,1678.606597,0.142605,...,-110.610146,-106.468140,-107.971695,98.786995,-55.476513,-142.753998,-32.750801,-76.657181,-32.856384,-189.476608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.445095,3.580498,12.819963,692204,6908,4018,2408,2325,938.480804,0.144515,...,-31.042624,-19.502481,-83.271576,-53.122066,-25.386417,72.956093,-7.632018,24.601561,-14.287844,43.360516
327,0.658623,4.684167,21.941421,691715,3259,2229,1782,2440,726.729216,0.142468,...,-5.182633,-21.588072,-116.423080,4.937935,-1.011641,14.841179,-30.999542,11.898485,-8.679675,3.121428
644,0.777467,5.432756,29.514836,1103406,19543,8748,5361,6343,1499.245582,0.132829,...,-20.025541,-13.606735,-50.646061,12.016315,-38.295330,-6.618800,13.547494,103.193474,-65.287102,-56.278809
608,0.411621,4.033740,16.271059,698187,5260,2424,1786,2215,742.820987,0.158994,...,29.394262,-30.534182,-114.291298,-31.632484,-26.607752,-3.680860,-17.245800,-1.621570,0.704188,-29.887348


In [17]:
y_test

763    1
299    0
897    2
341    0
40     0
      ..
317    0
327    0
644    1
608    0
343    0
Name: Class, Length: 212, dtype: int64

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

train_X = sc.fit_transform(x_train)
test_X = sc.transform(x_test)

In [19]:
train_X

array([[-0.50228912, -0.81817418, -0.76199226, ...,  0.41792913,
         0.122379  , -0.02866471],
       [-0.26438955,  0.78385119,  0.60212582, ...,  1.19322685,
         0.62439166,  0.28777847],
       [ 0.38854633,  0.99740058,  0.85457149, ...,  0.07805029,
        -0.48224901,  0.12868317],
       ...,
       [-0.12323499, -0.50349151, -0.56781741, ..., -0.68554179,
         1.8504912 ,  3.397396  ],
       [-0.52116737, -0.10516575, -0.27030781, ..., -0.22710932,
        -0.4377369 , -0.59765663],
       [-0.42752241,  0.52341473,  0.316734  , ...,  0.14131427,
         1.01347432,  0.85933342]])

In [20]:
test_X

array([[ 0.41596014,  1.94511674,  2.17533471, ..., -0.19718344,
         0.29179957, -0.03933211],
       [-0.51237198, -0.85951314, -0.78482014, ..., -0.11423094,
         0.32648398,  0.04595595],
       [ 0.3442099 ,  0.80161179,  0.62248808, ...,  1.29385271,
         3.41214954,  2.47424573],
       ...,
       [ 0.03226382,  0.43400595,  0.22445442, ...,  1.50866125,
        -0.79875979, -0.81313101],
       [-0.48006229, -0.28418946, -0.4111706 , ...,  0.10546388,
         0.10837805, -0.44102528],
       [-0.76512721, -0.98733922, -0.85146924, ...,  0.33592999,
         0.52610889,  0.36698329]])

In [21]:
#Decision Tree Classifier
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier
#Assign model with Decision Tree classifier


model_dt = DecisionTreeClassifier(max_depth=6)

model_dt.fit(x_train, y_train)

joblib.dump(model_dt,"./decision_tree_model/1.txt")

#predicting the traget variable using testing variables
y_pred1 = model_dt.predict(x_test)
#Results

print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("Train Accuracy:",model_dt.score(x_train, y_train))
print("Test Accuracy:",model_dt.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred1, pos_label='positive', average='micro')) # true positive rate, Sensitivity
print("F2 Score: ",metrics.fbeta_score(y_test, y_pred1, pos_label='positive', average='micro', beta=2.0))
print("F1 Score: ",metrics.f1_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred1))

Decision Tree Results
Decision Tree Accuracy:  77.83018867924528 %
Train Accuracy: 0.9361702127659575
Test Accuracy: 0.7783018867924528
Precision Score:  0.7783018867924528
Recall Score:  0.7783018867924528
F2 Score:  0.7783018867924528
F1 Score:  0.7783018867924528
Confusion Matrix: 
[[109  17   7]
 [ 11  34   1]
 [  9   2  22]]


In [22]:
#Decision Tree Classifier 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics

# Define the hyperparameters to search over
param_grid = {
    'max_depth': [2, 4, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

# Create a decision tree classifier object
dt = DecisionTreeClassifier()

#Assign model with Decision Tree classifier
model_dt = DecisionTreeClassifier()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model_dt, param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)


joblib.dump(grid_search,"./decision_tree_tunned_model/1.txt")

#predicting the traget variable using testing variables
y_pred1 = grid_search.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("Train Accuracy:",grid_search.score(x_train, y_train))
print("Test Accuracy:",grid_search.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred1, pos_label='positive', average='micro')) # true positive rate, Sensitivity
print("F2 Score: ",metrics.fbeta_score(y_test, y_pred1, pos_label='positive', average='micro', beta=2.0))
print("F1 Score: ",metrics.f1_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred1))

Best hyperparameters: {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 5, 'splitter': 'random'}
Best accuracy: 0.7742220675252349
Decision Tree Results
Decision Tree Accuracy:  80.66037735849056 %
Train Accuracy: 0.8475177304964538
Test Accuracy: 0.8066037735849056
Precision Score:  0.8066037735849056
Recall Score:  0.8066037735849056
F2 Score:  0.8066037735849056
F1 Score:  0.8066037735849056
Confusion Matrix: 
[[119   7   7]
 [ 13  32   1]
 [ 12   1  20]]


In [23]:
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics
import warnings
warnings.filterwarnings('ignore')

In [24]:
#Random forest classiffier
# df = df.drop('Label',axis=1)
X = df.iloc[:, :-1]  # selects all rows (:) and all but the last column (:-1) of the DataFrame df.
Y = df.iloc[:, -1]    #  selects all rows (:) and the last column (-1) of the DataFrame df.

# train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)
# Train a Random Forest classifier on the training set

In [25]:
import numpy as np
np.unique(y_train)

array([0, 1, 2], dtype=int64)

In [26]:
model2 = RandomForestClassifier(n_estimators=80, random_state=42)
model2.fit(x_train, y_train)
joblib.dump(model2,"./random_forest_model/1.txt")
y_pred2 = model2.predict(x_test)


print("Random Forest Clasifier")
print("Random Forest Accuracy: ",accuracy_score(y_test, y_pred2)*100,"%")
print("Train Accuracy:",model2.score(x_train, y_train))
print("Test Accuracy:",model2.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred2, average='macro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred2, average='macro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred2, average='macro'))
print("Confusion Matrix: ")
train_cm = confusion_matrix(y_test, y_pred2)
print(train_cm)

Random Forest Clasifier
Random Forest Accuracy:  84.43396226415094 %
Train Accuracy: 1.0
Test Accuracy: 0.8443396226415094
Precision Score:  0.8123216422719323
Recall Score:  0.7949929971988795
F1 Score:  0.7958260120014587
Confusion Matrix: 
[[124   3   9]
 [ 12  33   3]
 [  6   0  22]]


In [19]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 4, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

model_rf = RandomForestClassifier()


# Perform grid search to find the best hyperparameters
grid_search_rf = GridSearchCV(model_rf, param_grid, cv=5)
grid_search_rf.fit(x_train, y_train)


print("Best hyperparameters: ", grid_search_rf.best_params_)
print("Best score: ", grid_search_rf.best_score_)

joblib.dump(grid_search_rf,"./cpds/random_forest_tunned_model/1.txt")

y_pred2 = grid_search_rf.predict(x_test)
print("Random Forest Clasifier")
print("Train Accuracy:",grid_search_rf.score(x_train, y_train))
print("Test Accuracy:",grid_search_rf.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred2, pos_label='po  sitive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred2,))

KeyboardInterrupt: 

In [27]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors = 13)
model_knn.fit(x_train, y_train)


joblib.dump(model_knn,"./knn_model/1.txt")
y_pred3 = model_knn.predict(x_test)
print("KNN")
print("Train Accuracy:",model_knn.score(x_train, y_train))
print("Test Accuracy:",model_knn.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred3,))

KNN
Train Accuracy: 0.7659574468085106
Test Accuracy: 0.7594339622641509
Precision Score:  0.7594339622641509
Recall Score:  0.7594339622641509
F1 Score:  0.7594339622641509
Confusion Matrix: 
[[118   6  12]
 [ 18  26   4]
 [ 11   0  17]]


In [28]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier

# Define the parameter grid to search over
param_grid = {'n_neighbors': [5, 7, 13, 17, 23],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

# Create a KNeighborsClassifier object
knn = KNeighborsClassifier()

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search_knn = GridSearchCV(knn, param_grid, cv=5)

# Fit the grid search object to the training data
#model_knn.fit(x_train, y_train)
grid_search_knn.fit(x_train, y_train)

# Print the best hyperparameters and their corresponding score
print("Best hyperparameters: ", grid_search_knn.best_params_)
print("Best score: ", grid_search_knn.best_score_)

joblib.dump(grid_search_knn,"./knn_model_tunned/1.txt")

y_pred3 = grid_search_knn.predict(x_test)

print("KNN")
print("Train Accuracy:",grid_search_knn.score(x_train, y_train))
print("Test Accuracy:",grid_search_knn.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred3,))
# print("ROC curve ",metrics.plot_roc_curve(grid_search_knn,x_test, y_test))

Best hyperparameters:  {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
Best score:  0.7635851026801254
KNN
Train Accuracy: 0.8274231678486997
Test Accuracy: 0.7783018867924528
Precision Score:  0.7783018867924528
Recall Score:  0.7783018867924528
F1 Score:  0.7783018867924528
Confusion Matrix: 
[[120   3  13]
 [ 19  25   4]
 [  8   0  20]]


In [22]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Define SVM models with different kernels
svc_models = [
    SVC(kernel='linear'),
    SVC(kernel='poly', degree=2),
    SVC(kernel='poly', degree=3),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid')
]

# Train and evaluate SVM models
for svc in svc_models:
    svc.fit(x_train, y_train)
    accuracy = svc.score(x_test, y_test)
    print(f"{svc.kernel} kernel accuracy: {accuracy:.3f}")
    
    
    
    

In [30]:
my_test = [1.340523158, 8.990120269, 80.82226245, 672431, 6754, 4916, 4768, 5535, 912.6672699, 0.134691156, 0.111195749, 0.012364495, 3239, 1887, 1072, 387, 136, 39, 10, 3, 1, 2]
# 0.036231864	0.762807335	0.581875031	down	0	0	0	0	0	712233	2567	939	459	290
my_test_2= [5.335390625, 7.725038413, 59.67621849, 381548, 59452, 45020, 33297, 41230]

arr = np.array(my_test)
arr = arr.reshape(1,-1)

In [29]:
import joblib

# Save the trained model
joblib.dump(model2, './cpds/random_forest_model/1.txt')

['./cpds/random_forest_model/1.txt']

In [30]:
def extract_features_1(optical_flow,frame1,frame2, num_bins=10):
    # Calculate the mean, standard deviation, and variance of the optical flow image
    

    hsv = np.zeros((frame1.shape[0], frame1.shape[1], 3), dtype=np.uint8)
    
    hsv[...,1] = 255
    mag, ang = cv2.cartToPolar(optical_flow[...,0], optical_flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
       
    optical_flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    of_image = cv2.cvtColor(optical_flow_image, cv2.COLOR_BGR2GRAY)

    mean = np.mean(of_image)
    std = np.std(of_image)
    var = np.var(of_image)
    # Calculate the histogram of the optical flow image
    hist, bins = np.histogram(of_image, bins=num_bins, range=(-10, 10))
    hist = hist.tolist()
    hist.pop(0)
    hist.pop(1)
    hist.pop(2)
    hist.pop(3)
    hist.pop(4)

    
    h=np.unique(extract_hog_features(of_image))
    sum_hog=np.sum(h)
    mean_hog=np.mean(h)
    std_hog=np.std(h)
    var_hog=np.var(h)
    hog_hist=np.zeros(10)
    for i in h:
        if i>0.0 and i<0.1:
            hog_hist[0]+=1
        elif i>0.1 and i<0.2:
            hog_hist[1]+=1
        elif i>0.2 and i<0.3:
            hog_hist[2]+=1
        elif i>0.3 and i<0.4:
            hog_hist[3]+=1
        elif i>0.4 and i<0.5:
            hog_hist[4]+=1
        elif i>0.5 and i<0.6:
            hog_hist[5]+=1
        elif i>0.6 and i<0.7:
            hog_hist[6]+=1
        elif i>0.7 and i<0.8:
            hog_hist[7]+=1
        elif i>0.8 and i<0.9:
            hog_hist[8]+=1
        else:
            hog_hist[9]+=1
    hog_hist=hog_hist.tolist()
    
#     print(hist)
    


    return [mean, std, var] + hist + [sum_hog, mean_hog, std_hog, var_hog] + hog_hist

In [31]:
def extract_features_2(frame2):
    bgSubtractor = cv2.createBackgroundSubtractorMOG2()
    fgMask = bgSubtractor.apply(frame2)

    # Apply morphological operations to reduce noise and fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    #initialise sift descriptor
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(fgMask, None)
    # sift_image = cv2.drawKeypoints(gray, keypoints, img)
    try:
        descriptors=descriptors.T
        pca = PCA(n_components=0.95)
        # Fit the transformer to the data and transform the data
        descriptors = pca.fit_transform(descriptors)
        # Reshape the descriptors array into a 1D array
        descriptors_1d = descriptors.reshape(-1)
        descriptors_1d = descriptors_1d[:128]
    except:
        descriptors_1d=np.zeros(128)
    descriptors_1d=descriptors_1d.tolist()
    return descriptors_1d


In [33]:
import cv2
import numpy as np
from skimage.feature import hog


def extract_hog_features(image):
    # define the HOG parameters
    orientations = 9
    pixels_per_cell = (16, 16)
    cells_per_block = (2, 2)
    
    # compute the HOG features for the image
    features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=False, transform_sqrt=True)
    print('\n')
    # return the features as a 1D NumPy array
    return features.ravel()

In [49]:
import cv2
import numpy as np
import joblib

# Load the saved Random Forest model
model = joblib.load('./random_forest_model/1.txt')
csv_path_1 = "./test_video.csv"
# Define the video capture object
cap = cv2.VideoCapture('./dataset/001.mp4')
with open(csv_path_1, 'w', newline='') as file:
    writer = csv.writer(file)
    ret, frame1 = cap.read()
    i = 0
    count_back = 0
    count_down = 0
    count_follow = 0
        # Loop through the rest of the frames
    while True:

            # Read the next frame
        ret, frame2 = cap.read()

          # If there are no more frames, break out of the loop
        if not ret:
        	break

        # Convert the frame to grayscale
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        # Compute the optical flow using the Farneback method
        flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        resArr_1 = extract_features_1(flow,frame1,frame2)
        resArr_2 = extract_features_2(frame2)
        resArr=resArr_1+resArr_2
        frame1=frame2

        # Reshape the features into a 2D array
        X = np.array(resArr).reshape(1, -1)

        # Use the loaded Random Forest model to predict the label
        label = model.predict(X)[0]
        # print(type(int(label)))
        # Display the label on the frame
        #cv2.putText(gray1, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the resulting frame
        #cv2.imshow('frame', frame1)
        if int(label) == 0:
            count_back += 1
            writer.writerow("back")
            # print("back")
        elif int(label) == 1:
            count_down +=1
            writer.writerow("down")
            # print("down")
        elif int(label) == 2:
            count_follow +=1
            writer.writerow("follow")
            # print("follow through")

        print('Count back : ',count_back)
        print('Count Down : ',count_down)
        print('Count Follow : ',count_follow)

#     if count_back > count_down:
#         print("back")
#     else:
#         print("down")

    # Check for quit key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

# if (count_back>count_down and count_back>count_follow):
#     print("Video Class: Back")
# if (count_down>count_back and count_down>count_follow):
#     print("Video Class: Down")
# if (count_follow>count_down and count_back<count_follow):
#     print("Video Class: Follow")




Count back :  1
Count Down :  0
Count Follow :  0


Count back :  2
Count Down :  0
Count Follow :  0


Count back :  2
Count Down :  1
Count Follow :  0


Count back :  3
Count Down :  1
Count Follow :  0


Count back :  4
Count Down :  1
Count Follow :  0


Count back :  5
Count Down :  1
Count Follow :  0


Count back :  6
Count Down :  1
Count Follow :  0


Count back :  7
Count Down :  1
Count Follow :  0


Count back :  8
Count Down :  1
Count Follow :  0


Count back :  9
Count Down :  1
Count Follow :  0


Count back :  10
Count Down :  1
Count Follow :  0


Count back :  11
Count Down :  1
Count Follow :  0


Count back :  12
Count Down :  1
Count Follow :  0


Count back :  13
Count Down :  1
Count Follow :  0


Count back :  14
Count Down :  1
Count Follow :  0


Count back :  14
Count Down :  2
Count Follow :  0


Count back :  15
Count Down :  2
Count Follow :  0


Count back :  16
Count Down :  2
Count Follow :  0


Count back :  17
Count Down :  2
Count Follow :  0


C

In [35]:
import cv2
import numpy as np
import joblib

# Load the saved Random Forest model
model = joblib.load('./cpds/random_forest_model/1.txt')

# Define the video capture object
cap = cv2.VideoCapture('./cpds/dataset/follow_022.mp4')
ret, frame1 = cap.read()
i = 0
count_back = 0
count_down = 0
count_follow = 0
    # Loop through the rest of the frames
while True:
        
        # Read the next frame
    ret, frame2 = cap.read()
        
      # If there are no more frames, break out of the loop
    if not ret:
    	break

    # Convert the frame to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Compute the optical flow using the Farneback method
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    resArr_1 = extract_features_1(flow,frame1,frame2)
    resArr_2 = extract_features_2(frame2)
    resArr=resArr_1+resArr_2
    frame1=frame2
    
    # Reshape the features into a 2D array
    X = np.array(resArr).reshape(1, -1)
    
    # Use the loaded Random Forest model to predict the label
    label = model.predict(X)[0]
    # print(type(int(label)))
    # Display the label on the frame
    #cv2.putText(gray1, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    #cv2.imshow('frame', frame1)
    if int(label) == 0:
        count_back += 1
    elif int(label) == 1:
        count_down +=1
    elif int(label) == 2:
        count_follow +=1
        
    print('Count back : ',count_back)
    print('Count Down : ',count_down)
    print('Count Follow : ',count_follow)
    
#     if count_back > count_down:
#         print("back")
#     else:
#         print("down")

    # Check for quit key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

print(max(count_back, count_down, count_follow))



Count back :  0
Count Down :  0
Count Follow :  1


Count back :  0
Count Down :  0
Count Follow :  2


Count back :  0
Count Down :  0
Count Follow :  3


Count back :  0
Count Down :  0
Count Follow :  4


Count back :  0
Count Down :  0
Count Follow :  5


Count back :  0
Count Down :  0
Count Follow :  6


Count back :  0
Count Down :  0
Count Follow :  7


Count back :  0
Count Down :  0
Count Follow :  8


Count back :  0
Count Down :  0
Count Follow :  9


Count back :  0
Count Down :  0
Count Follow :  10


Count back :  0
Count Down :  0
Count Follow :  11


Count back :  0
Count Down :  0
Count Follow :  12


Count back :  0
Count Down :  0
Count Follow :  13


Count back :  1
Count Down :  0
Count Follow :  13


Count back :  1
Count Down :  0
Count Follow :  14


Count back :  1
Count Down :  0
Count Follow :  15


Count back :  1
Count Down :  0
Count Follow :  16


Count back :  1
Count Down :  0
Count Follow :  17


Count back :  1
Count Down :  0
Count Follow :  18




In [39]:
import cv2
import numpy as np
import joblib

# Load the saved Random Forest model
model = joblib.load('./cpds/random_forest_model/1.txt')

# Define the video capture object
cap = cv2.VideoCapture('./cpds/dataset/down_001.mp4')
ret, frame1 = cap.read()
i = 0
count_back = 0
count_down = 0
count_follow = 0
    # Loop through the rest of the frames
while True:
        
        # Read the next frame
    ret, frame2 = cap.read()
        
      # If there are no more frames, break out of the loop
    if not ret:
    	break

    # Convert the frame to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Compute the optical flow using the Farneback method
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    resArr_1 = extract_features_1(flow,frame1,frame2)
    resArr_2 = extract_features_2(frame2)
    resArr=resArr_1+resArr_2
    frame1=frame2
    
    # Reshape the features into a 2D array
    X = np.array(resArr).reshape(1, -1)
    
    # Use the loaded Random Forest model to predict the label
    label = model.predict(X)[0]
    # print(type(int(label)))
    # Display the label on the frame
    #cv2.putText(gray1, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    #cv2.imshow('frame', frame1)
    if int(label) == 0:
        count_back += 1
    elif int(label) == 1:
        count_down +=1
    elif int(label) == 2:
        count_follow +=1
        
    print('Count back : ',count_back)
    print('Count Down : ',count_down)
    print('Count Follow : ',count_follow)
    
#     if count_back > count_down:
#         print("back")
#     else:
#         print("down")

    # Check for quit key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

print(max(count_back, count_down, count_follow))



Count back :  0
Count Down :  1
Count Follow :  0


Count back :  0
Count Down :  2
Count Follow :  0


Count back :  0
Count Down :  3
Count Follow :  0


Count back :  0
Count Down :  4
Count Follow :  0


Count back :  0
Count Down :  5
Count Follow :  0


Count back :  0
Count Down :  6
Count Follow :  0


Count back :  0
Count Down :  7
Count Follow :  0


Count back :  0
Count Down :  8
Count Follow :  0


Count back :  0
Count Down :  9
Count Follow :  0


Count back :  0
Count Down :  10
Count Follow :  0


Count back :  0
Count Down :  11
Count Follow :  0


Count back :  0
Count Down :  12
Count Follow :  0


Count back :  0
Count Down :  13
Count Follow :  0


Count back :  0
Count Down :  14
Count Follow :  0


Count back :  0
Count Down :  15
Count Follow :  0


Count back :  0
Count Down :  16
Count Follow :  0


Count back :  0
Count Down :  17
Count Follow :  0


Count back :  0
Count Down :  18
Count Follow :  0


Count back :  0
Count Down :  19
Count Follow :  0




In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the CSV file with extracted features
csv_file_path = './cpds/features.csv'
df = pd.read_csv(csv_file_path)

X = df.drop('Label',axis=1)
y = df['Label']
# print👍

new_col_values = []

# loop through rows of DataFrame
for row in df.iterrows():
    # print(row[1]['Label'])
    if row[1]['Label']=="down":
        new_col_values.append(1)
    else:
        new_col_values.append(0)

# add new_col to DataFrame
df['Class'] = new_col_values

df.to_csv(csv_file_path)

# print updated DataFrame
print(df)

      Unnamed: 0      Mean  Standard Deviation   Variance   Label    Bin 5  \
0              0  0.556674            4.696250  22.054761    back  1094022   
1              1  0.359380            3.186641  10.154681    back  1114348   
2              2  0.741009            4.105302  16.853508    back  1072170   
3              3  0.795212            5.239447  27.451801    back  1078831   
4              4  0.414621            3.618001  13.089933    back  1111613   
...          ...       ...                 ...        ...     ...      ...   
1053        1053  1.349114            4.987656  24.876710  follow  1002155   
1054        1054  0.767247            3.495845  12.220935  follow  1046373   
1055        1055  0.966338            3.527668  12.444442  follow  1016054   
1056        1056  0.738081            3.138769   9.851868  follow  1043169   
1057        1057  2.213606            5.615943  31.538815  follow   868385   

      Bin 6  Bin 7  Bin 8  Bin 9  ...   SIFT 119   SIFT 120    

In [2]:

# df.drop('Unnamed',axis=1)
df = df.drop(df.columns[0],axis=1)
df = df.drop('Label',axis=1)
X = df.iloc[:, :-1]  # selects all rows (:) and all but the last column (:-1) of the DataFrame df.
Y = df.iloc[:, -1]    #  selects all rows (:) and the last column (-1) of the DataFrame df.

# train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=5)

In [3]:
X

,Mean,Standard Deviation,Variance,Bin 5,Bin 6,Bin 7,Bin 8,Bin 9,Sum HOG,Mean HOG,...,SIFT 118,SIFT 119,SIFT 120,SIFT 121,SIFT 122,SIFT 123,SIFT 124,SIFT 125,SIFT 126,SIFT 127
0,0.556674,4.696250,22.054761,1094022,31458,15121,7711,6220,2814.904485,0.152868,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.359380,3.186641,10.154681,1114348,23904,10572,4861,4056,1880.101450,0.148472,...,-74.772041,53.019558,-91.951920,-35.534462,134.467514,-39.382416,13.225515,5.185749,-25.173859,-2.575677
2,0.741009,4.105302,16.853508,1072170,38711,16174,8883,8404,3193.181075,0.146449,...,119.858444,64.790573,-17.979828,-11.774525,10.488808,39.699814,30.242283,-12.214418,-21.354784,97.238396
3,0.795212,5.239447,27.451801,1078831,33555,14005,9516,9568,2813.931181,0.145536,...,-45.751472,-8.146722,23.674538,16.879667,-29.073320,-32.995296,-8.107094,10.847504,-168.261124,-165.720993
4,0.414621,3.618001,13.089933,1111613,22207,11422,6288,4955,1720.966500,0.143342,...,-28.151375,20.717064,-37.631458,-11.912526,2.186476,-57.171062,12.862970,-19.574232,24.376343,33.111610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1.349114,4.987656,24.876710,1002155,53381,30298,18536,17274,5067.250268,0.138893,...,-5.987559,27.111052,69.208099,-64.069801,0.011178,18.976994,-21.876720,-63.722820,-223.607834,-112.845856
1054,0.767247,3.495845,12.220935,1046373,45812,24532,14780,13416,3089.014868,0.130531,...,8.201757,39.517273,42.473213,-129.972305,40.082336,25.547880,-0.518717,-135.653442,-0.696796,57.745838
1055,0.966338,3.527668,12.444442,1016054,54029,31988,19087,17610,4046.270988,0.134979,...,-127.677567,22.768131,-12.361745,-34.419037,-54.391846,54.337955,-134.791870,68.316101,-37.639439,-42.464138
1056,0.738081,3.138769,9.851868,1043169,51997,26083,13887,12107,3269.506477,0.129949,...,67.561134,24.446341,-24.754156,44.638222,7.681120,13.457706,66.544075,-33.429150,-5.936067,-14.595716


In [4]:
Y


0       0
1       0
2       0
3       0
4       0
       ..
1053    0
1054    0
1055    0
1056    0
1057    0
Name: Class, Length: 1058, dtype: int64

In [5]:
x_train

,Mean,Standard Deviation,Variance,Bin 5,Bin 6,Bin 7,Bin 8,Bin 9,Sum HOG,Mean HOG,...,SIFT 118,SIFT 119,SIFT 120,SIFT 121,SIFT 122,SIFT 123,SIFT 124,SIFT 125,SIFT 126,SIFT 127
281,0.395749,2.993559,8.961397,692638,4804,3773,3559,3741,718.828264,0.142033,...,-14.364077,17.185566,4.719944,-20.514751,-6.963824,27.971216,-21.066498,21.718739,1.722710,-0.640827
830,0.565631,6.114240,37.383935,697335,4999,2594,1542,1686,579.192828,0.143471,...,-23.141712,35.656250,-25.227655,-6.477972,8.991025,-188.709259,-4.774416,79.631378,38.242477,21.802788
683,1.031885,6.530226,42.643851,677264,7990,5239,3734,3830,1232.412288,0.140510,...,41.050419,-0.057453,50.055229,-63.717213,-14.580037,87.122253,-4.349473,-3.669290,-42.261986,10.519012
871,0.645607,6.193820,38.363402,696932,5960,2094,1191,2059,496.504969,0.138264,...,-2.855215,36.042580,34.075413,-18.208763,-24.695837,-13.331373,60.803856,-43.332256,30.979496,-24.738859
595,0.149964,1.770950,3.136264,706812,2856,2157,1090,834,440.807715,0.168633,...,-5.605232,-20.277100,-15.782651,13.718862,-11.134361,-0.968225,-13.858082,-135.757111,-42.307270,9.925239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,2.362716,6.395596,40.903648,832399,128542,67458,38195,33420,8949.271374,0.126329,...,83.593719,-68.629028,-52.677101,-28.887522,25.744452,15.880880,-2.844508,-392.439697,-108.001328,-45.398540
73,0.979023,5.981757,35.781414,1086071,29942,11212,5695,5855,2800.138127,0.150618,...,153.133026,45.206825,79.063889,-89.260727,6.035251,-36.186443,-164.663254,-163.252563,53.850796,-97.443893
998,0.666427,3.606549,13.007192,1066480,43049,19533,10962,9229,3251.677323,0.141790,...,-43.079197,-108.933044,-36.067825,-56.453537,1.692349,-110.791862,36.061924,-60.707550,127.437180,242.351257
206,0.382268,4.382471,19.206053,697982,5193,2961,1828,2474,587.808965,0.144923,...,28.080463,-20.961449,-39.358562,51.183037,-43.236012,78.299339,-35.319405,-26.463886,-39.023876,-40.996368


In [6]:
y_train

281     0
830     1
683     1
871     1
595     0
       ..
1032    0
73      0
998     0
206     0
867     1
Name: Class, Length: 846, dtype: int64

In [7]:
x_test

,Mean,Standard Deviation,Variance,Bin 5,Bin 6,Bin 7,Bin 8,Bin 9,Sum HOG,Mean HOG,...,SIFT 118,SIFT 119,SIFT 120,SIFT 121,SIFT 122,SIFT 123,SIFT 124,SIFT 125,SIFT 126,SIFT 127
763,1.051461,8.376339,70.163054,684665,5443,3950,4236,3962,764.386099,0.128167,...,-36.567074,-14.384775,-11.734731,37.541889,13.553376,-0.292490,-14.866241,-24.228504,14.047499,-1.397409
299,0.388549,2.913033,8.485760,693077,4127,3494,3348,4036,622.158726,0.139435,...,6.850658,9.915338,-9.978570,-20.811613,5.730747,-7.889675,-14.599580,-18.032177,16.570675,4.651615
897,1.000225,6.148837,37.808200,1097417,15494,7525,5695,7542,1818.978238,0.140625,...,-191.719376,-260.093048,42.887104,-141.701385,-129.961090,-232.116409,232.042786,87.147858,241.042679,176.877167
341,0.280698,2.752521,7.576374,697887,5483,3004,2249,3177,598.177976,0.146720,...,2.614320,-40.131428,-15.332307,0.684482,4.942315,-18.190737,-36.530193,-15.180065,24.401747,4.227867
40,0.522675,3.299020,10.883532,1111035,17915,10108,5928,4734,1678.606597,0.142605,...,-110.610146,-106.468140,-107.971695,98.786995,-55.476513,-142.753998,-32.750801,-76.657181,-32.856384,-189.476608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.445095,3.580498,12.819963,692204,6908,4018,2408,2325,938.480804,0.144515,...,-31.042624,-19.502481,-83.271576,-53.122066,-25.386417,72.956093,-7.632018,24.601561,-14.287844,43.360516
327,0.658623,4.684167,21.941421,691715,3259,2229,1782,2440,726.729216,0.142468,...,-5.182633,-21.588072,-116.423080,4.937935,-1.011641,14.841179,-30.999542,11.898485,-8.679675,3.121428
644,0.777467,5.432756,29.514836,1103406,19543,8748,5361,6343,1499.245582,0.132829,...,-20.025541,-13.606735,-50.646061,12.016315,-38.295330,-6.618800,13.547494,103.193474,-65.287102,-56.278809
608,0.411621,4.033740,16.271059,698187,5260,2424,1786,2215,742.820987,0.158994,...,29.394262,-30.534182,-114.291298,-31.632484,-26.607752,-3.680860,-17.245800,-1.621570,0.704188,-29.887348


In [8]:
y_test

763    1
299    0
897    0
341    0
40     0
      ..
317    0
327    0
644    1
608    0
343    0
Name: Class, Length: 212, dtype: int64

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

train_X = sc.fit_transform(x_train)
test_X = sc.transform(x_test)

In [10]:
train_X

array([[-0.50228912, -0.81817418, -0.76199226, ...,  0.41792913,
         0.122379  , -0.02866471],
       [-0.26438955,  0.78385119,  0.60212582, ...,  1.19322685,
         0.62439166,  0.28777847],
       [ 0.38854633,  0.99740058,  0.85457149, ...,  0.07805029,
        -0.48224901,  0.12868317],
       ...,
       [-0.12323499, -0.50349151, -0.56781741, ..., -0.68554179,
         1.8504912 ,  3.397396  ],
       [-0.52116737, -0.10516575, -0.27030781, ..., -0.22710932,
        -0.4377369 , -0.59765663],
       [-0.42752241,  0.52341473,  0.316734  , ...,  0.14131427,
         1.01347432,  0.85933342]])

In [11]:
test_X

array([[ 0.41596014,  1.94511674,  2.17533471, ..., -0.19718344,
         0.29179957, -0.03933211],
       [-0.51237198, -0.85951314, -0.78482014, ..., -0.11423094,
         0.32648398,  0.04595595],
       [ 0.3442099 ,  0.80161179,  0.62248808, ...,  1.29385271,
         3.41214954,  2.47424573],
       ...,
       [ 0.03226382,  0.43400595,  0.22445442, ...,  1.50866125,
        -0.79875979, -0.81313101],
       [-0.48006229, -0.28418946, -0.4111706 , ...,  0.10546388,
         0.10837805, -0.44102528],
       [-0.76512721, -0.98733922, -0.85146924, ...,  0.33592999,
         0.52610889,  0.36698329]])

In [12]:
#Decision Tree Classifier
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier
#Assign model with Decision Tree classifier


model_dt = DecisionTreeClassifier(max_depth=6)

model_dt.fit(x_train, y_train)

joblib.dump(model_dt,"./cpds/decision_tree_model/1.txt")

#predicting the traget variable using testing variables
y_pred1 = model_dt.predict(x_test)
#Results

print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("Train Accuracy:",model_dt.score(x_train, y_train))
print("Test Accuracy:",model_dt.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred1, pos_label='positive', average='micro')) # true positive rate, Sensitivity
print("F2 Score: ",metrics.fbeta_score(y_test, y_pred1, pos_label='positive', average='micro', beta=2.0))
print("F1 Score: ",metrics.f1_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred1))

Decision Tree Results
Decision Tree Accuracy:  82.0754716981132 %
Train Accuracy: 0.9609929078014184
Test Accuracy: 0.8207547169811321
Precision Score:  0.8207547169811321
Recall Score:  0.8207547169811321
F2 Score:  0.8207547169811321
F1 Score:  0.8207547169811321
Confusion Matrix: 
[[146  20]
 [ 18  28]]


d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a 

In [13]:
#Decision Tree Classifier 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics

# Define the hyperparameters to search over
param_grid = {
    'max_depth': [2, 4, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

# Create a decision tree classifier object
dt = DecisionTreeClassifier()

#Assign model with Decision Tree classifier
model_dt = DecisionTreeClassifier()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model_dt, param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)


joblib.dump(grid_search,"./cpds/decision_tree_tunned_model/1.txt")

#predicting the traget variable using testing variables
y_pred1 = grid_search.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("Train Accuracy:",grid_search.score(x_train, y_train))
print("Test Accuracy:",grid_search.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred1, pos_label='positive', average='micro')) # true positive rate, Sensitivity
print("F2 Score: ",metrics.fbeta_score(y_test, y_pred1, pos_label='positive', average='micro', beta=2.0))
print("F1 Score: ",metrics.f1_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred1))

Best hyperparameters: {'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'random'}
Best accuracy: 0.8699547511312217
Decision Tree Results
Decision Tree Accuracy:  82.0754716981132 %
Train Accuracy: 0.8404255319148937
Test Accuracy: 0.8207547169811321
Precision Score:  0.8207547169811321
Recall Score:  0.8207547169811321
F2 Score:  0.8207547169811321
F1 Score:  0.8207547169811321
Confusion Matrix: 
[[140  26]
 [ 12  34]]


d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
d:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a 

In [14]:
from sklearn.metrics import accuracy_score
import joblib
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics
import warnings
warnings.filterwarnings('ignore')

In [15]:
#Random forest classiffier
# df = df.drop('Label',axis=1)
X = df.iloc[:, :-1]  # selects all rows (:) and all but the last column (:-1) of the DataFrame df.
Y = df.iloc[:, -1]    #  selects all rows (:) and the last column (-1) of the DataFrame df.

# train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)
# Train a Random Forest classifier on the training set

In [16]:
import numpy as np
np.unique(y_train)

array([0, 1], dtype=int64)

In [17]:
model2 = RandomForestClassifier(n_estimators=80, random_state=42)
model2.fit(x_train, y_train)
joblib.dump(model2,"./cpds/random_forest_model/1.txt")
y_pred2 = model2.predict(x_test)


print("Random Forest Clasifier")
print("Random Forest Accuracy: ",accuracy_score(y_test, y_pred2)*100,"%")
print("Train Accuracy:",model2.score(x_train, y_train))
print("Test Accuracy:",model2.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred2, average='macro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred2, average='macro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred2, average='macro'))
print("Confusion Matrix: ")
train_cm = confusion_matrix(y_test, y_pred2)
print(train_cm)

Random Forest Clasifier
Random Forest Accuracy:  90.56603773584906 %
Train Accuracy: 1.0
Test Accuracy: 0.9056603773584906
Precision Score:  0.898989898989899
Recall Score:  0.8211382113821137
F1 Score:  0.8515406162464986
Confusion Matrix: 
[[160   4]
 [ 16  32]]


In [198]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 4, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

model_rf = RandomForestClassifier()


# Perform grid search to find the best hyperparameters
grid_search_rf = GridSearchCV(model_rf, param_grid, cv=5)
grid_search_rf.fit(x_train, y_train)


print("Best hyperparameters: ", grid_search_rf.best_params_)
print("Best score: ", grid_search_rf.best_score_)

joblib.dump(grid_search_rf,"./cpds/random_forest_tunned_model/1.txt")

y_pred2 = grid_search_rf.predict(x_test)
print("Random Forest Clasifier")
print("Train Accuracy:",grid_search_rf.score(x_train, y_train))
print("Test Accuracy:",grid_search_rf.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred2, pos_label='po  sitive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred2,))

Best hyperparameters:  {'criterion': 'entropy', 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score:  0.9574382178907065
Random Forest Clasifier
Train Accuracy: 0.9929078014184397
Test Accuracy: 0.9528301886792453
Precision Score:  0.9528301886792453
Recall Score:  0.9528301886792453
F1 Score:  0.9528301886792453
Confusion Matrix: 
[[163   1]
 [  9  39]]


In [18]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors = 13)
model_knn.fit(x_train, y_train)


joblib.dump(model_knn,"./cpds/knn_model/1.txt")
y_pred3 = model_knn.predict(x_test)
print("KNN")
print("Train Accuracy:",model_knn.score(x_train, y_train))
print("Test Accuracy:",model_knn.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred3,))

KNN
Train Accuracy: 0.8581560283687943
Test Accuracy: 0.8632075471698113
Precision Score:  0.8632075471698113
Recall Score:  0.8632075471698113
F1 Score:  0.8632075471698113
Confusion Matrix: 
[[158   6]
 [ 23  25]]


In [19]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier

# Define the parameter grid to search over
param_grid = {'n_neighbors': [5, 7, 13, 17, 23],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

# Create a KNeighborsClassifier object
knn = KNeighborsClassifier()

# Perform a grid search over the parameter grid using 5-fold cross-validation
grid_search_knn = GridSearchCV(knn, param_grid, cv=5)

# Fit the grid search object to the training data
#model_knn.fit(x_train, y_train)
grid_search_knn.fit(x_train, y_train)

# Print the best hyperparameters and their corresponding score
print("Best hyperparameters: ", grid_search_knn.best_params_)
print("Best score: ", grid_search_knn.best_score_)

joblib.dump(grid_search_knn,"./cpds/knn_model_tunned/1.txt")

y_pred3 = grid_search_knn.predict(x_test)

print("KNN")
print("Train Accuracy:",grid_search_knn.score(x_train, y_train))
print("Test Accuracy:",grid_search_knn.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred3,))
# print("ROC curve ",metrics.plot_roc_curve(grid_search_knn,x_test, y_test))

Best hyperparameters:  {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
Best score:  0.8510755308040375
KNN
Train Accuracy: 1.0
Test Accuracy: 0.8773584905660378
Precision Score:  0.8773584905660378
Recall Score:  0.8773584905660378
F1 Score:  0.8773584905660378
Confusion Matrix: 
[[161   3]
 [ 23  25]]


In [20]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Define SVM models with different kernels
svc_models = [
    SVC(kernel='linear'),
    SVC(kernel='poly', degree=2),
    SVC(kernel='poly', degree=3),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid')
]

# Train and evaluate SVM models
for svc in svc_models:
    svc.fit(x_train, y_train)
    accuracy = svc.score(x_test, y_test)
    print(f"{svc.kernel} kernel accuracy: {accuracy:.3f}")
    
    
    
    

linear kernel accuracy: 0.830
poly kernel accuracy: 0.774
poly kernel accuracy: 0.774
rbf kernel accuracy: 0.774
sigmoid kernel accuracy: 0.774


In [21]:
import joblib

# Save the trained model
joblib.dump(model2, './cpds/random_forest_model/1.txt')

['./cpds/random_forest_model/1.txt']

In [24]:
def extract_features_1(optical_flow,frame1,frame2, num_bins=10):
    # Calculate the mean, standard deviation, and variance of the optical flow image
    

    hsv = np.zeros((frame1.shape[0], frame1.shape[1], 3), dtype=np.uint8)
    
    hsv[...,1] = 255
    mag, ang = cv2.cartToPolar(optical_flow[...,0], optical_flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
       
    optical_flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    of_image = cv2.cvtColor(optical_flow_image, cv2.COLOR_BGR2GRAY)

    mean = np.mean(of_image)
    std = np.std(of_image)
    var = np.var(of_image)
    # Calculate the histogram of the optical flow image
    hist, bins = np.histogram(of_image, bins=num_bins, range=(-10, 10))
    hist = hist.tolist()
    hist.pop(0)
    hist.pop(1)
    hist.pop(2)
    hist.pop(3)
    hist.pop(4)

    
    h=np.unique(extract_hog_features(of_image))
    sum_hog=np.sum(h)
    mean_hog=np.mean(h)
    std_hog=np.std(h)
    var_hog=np.var(h)
    hog_hist=np.zeros(10)
    for i in h:
        if i>0.0 and i<0.1:
            hog_hist[0]+=1
        elif i>0.1 and i<0.2:
            hog_hist[1]+=1
        elif i>0.2 and i<0.3:
            hog_hist[2]+=1
        elif i>0.3 and i<0.4:
            hog_hist[3]+=1
        elif i>0.4 and i<0.5:
            hog_hist[4]+=1
        elif i>0.5 and i<0.6:
            hog_hist[5]+=1
        elif i>0.6 and i<0.7:
            hog_hist[6]+=1
        elif i>0.7 and i<0.8:
            hog_hist[7]+=1
        elif i>0.8 and i<0.9:
            hog_hist[8]+=1
        else:
            hog_hist[9]+=1
    hog_hist=hog_hist.tolist()
    
#     print(hist)
    


    return [mean, std, var] + hist + [sum_hog, mean_hog, std_hog, var_hog] + hog_hist

In [22]:
def extract_features_2(frame2):
    bgSubtractor = cv2.createBackgroundSubtractorMOG2()
    fgMask = bgSubtractor.apply(frame2)

    # Apply morphological operations to reduce noise and fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    #initialise sift descriptor
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(fgMask, None)
    # sift_image = cv2.drawKeypoints(gray, keypoints, img)
    try:
        descriptors=descriptors.T
        pca = PCA(n_components=0.95)
        # Fit the transformer to the data and transform the data
        descriptors = pca.fit_transform(descriptors)
        # Reshape the descriptors array into a 1D array
        descriptors_1d = descriptors.reshape(-1)
        descriptors_1d = descriptors_1d[:128]
    except:
        descriptors_1d=np.zeros(128)
    descriptors_1d=descriptors_1d.tolist()
    return descriptors_1d


In [27]:
import cv2
import numpy as np
import joblib

# Load the saved Random Forest model
model = joblib.load('./cpds/random_forest_model/1.txt')

# Define the video capture object
cap = cv2.VideoCapture('./cpds/dataset/back_009.mp4')
ret, frame1 = cap.read()
i = 0
count_down = 0
count_other = 0
    # Loop through the rest of the frames
while True:
        
        # Read the next frame
    ret, frame2 = cap.read()
        
      # If there are no more frames, break out of the loop
    if not ret:
    	break

    # Convert the frame to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Compute the optical flow using the Farneback method
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    resArr_1 = extract_features_1(flow,frame1,frame2)
    resArr_2 = extract_features_2(frame2)
    resArr=resArr_1+resArr_2
    frame1=frame2
    
    # Reshape the features into a 2D array
    X = np.array(resArr).reshape(1, -1)
    
    # Use the loaded Random Forest model to predict the label
    label = model.predict(X)[0]
    # print(type(int(label)))
    # Display the label on the frame
    #cv2.putText(gray1, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    #cv2.imshow('frame', frame1)
    if int(label) == 1:
        count_down += 1
    elif int(label) == 0:
        count_other +=1
        
    print('Count Down : ',count_down)
    print('Count other : ',count_other)
    
#     if count_back > count_down:
#         print("back")
#     else:
#         print("down")

    # Check for quit key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

if count_down > count_other:
    print("Final Result : down")
else:
    print("Final Result : other")



Count Down :  0
Count other :  1


Count Down :  0
Count other :  2


Count Down :  0
Count other :  3


Count Down :  0
Count other :  4


Count Down :  0
Count other :  5


Count Down :  0
Count other :  6


Count Down :  0
Count other :  7


Count Down :  0
Count other :  8


Count Down :  0
Count other :  9


Count Down :  0
Count other :  10


Count Down :  0
Count other :  11


Count Down :  0
Count other :  12


Count Down :  0
Count other :  13


Count Down :  0
Count other :  14


Count Down :  0
Count other :  15


Count Down :  0
Count other :  16


Count Down :  0
Count other :  17


Count Down :  0
Count other :  18


Count Down :  0
Count other :  19


Count Down :  0
Count other :  20


Count Down :  0
Count other :  21


Count Down :  0
Count other :  22


Count Down :  0
Count other :  23


Count Down :  0
Count other :  24


Count Down :  0
Count other :  25


KeyboardInterrupt: 

In [28]:
import cv2
import numpy as np
import joblib

# Load the saved Random Forest model
model = joblib.load('./cpds/random_forest_model/1.txt')

# Define the video capture object
cap = cv2.VideoCapture('./cpds/dataset/down_003.mp4')
ret, frame1 = cap.read()
i = 0
count_down = 0
count_other = 0
    # Loop through the rest of the frames
while True:
        
        # Read the next frame
    ret, frame2 = cap.read()
        
      # If there are no more frames, break out of the loop
    if not ret:
    	break

    # Convert the frame to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # Compute the optical flow using the Farneback method
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    resArr_1 = extract_features_1(flow,frame1,frame2)
    resArr_2 = extract_features_2(frame2)
    resArr=resArr_1+resArr_2
    frame1=frame2
    
    # Reshape the features into a 2D array
    X = np.array(resArr).reshape(1, -1)
    
    # Use the loaded Random Forest model to predict the label
    label = model.predict(X)[0]
    # print(type(int(label)))
    # Display the label on the frame
    #cv2.putText(gray1, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    #cv2.imshow('frame', frame1)
    if int(label) == 1:
        count_down += 1
    elif int(label) == 0:
        count_other +=1
        
    print('Count back : ',count_down)
    print('Count Down : ',count_other)
    
#     if count_back > count_down:
#         print("back")
#     else:
#         print("down")

    # Check for quit key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

if count_down > count_other:
    print("Final Result : down")
else:
    print("Final Result : other")



Count back :  1
Count Down :  0


Count back :  2
Count Down :  0


Count back :  3
Count Down :  0


Count back :  4
Count Down :  0


Count back :  5
Count Down :  0


Count back :  6
Count Down :  0


Count back :  7
Count Down :  0


Count back :  8
Count Down :  0


Count back :  9
Count Down :  0


Count back :  10
Count Down :  0


Count back :  11
Count Down :  0


Count back :  12
Count Down :  0


Count back :  13
Count Down :  0


Count back :  14
Count Down :  0


Count back :  15
Count Down :  0


Count back :  16
Count Down :  0


Count back :  17
Count Down :  0


Count back :  18
Count Down :  0


Count back :  19
Count Down :  0


Count back :  20
Count Down :  0


Count back :  21
Count Down :  0


Count back :  22
Count Down :  0


Count back :  23
Count Down :  0


Count back :  24
Count Down :  0


Count back :  25
Count Down :  0


Count back :  26
Count Down :  0


Count back :  27
Count Down :  0


Count back :  28
Count Down :  0


Count back :  29
Count Down